In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import (
    RobertaTokenizer, 
    RobertaForSequenceClassification,
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset
from tqdm import tqdm
from itertools import permutations
import warnings
warnings.filterwarnings('ignore')


c:\Anaconda3\envs\daycon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_label_mappings():
    """24가지 순열에 대한 라벨 매핑 생성"""
    # 가능한 모든 순열 생성 (4! = 24개)
    all_permutations = list(permutations([0, 1, 2, 3]))
    
    # 순열 → 라벨 번호 매핑
    perm_to_label = {perm: idx for idx, perm in enumerate(all_permutations)}
    
    # 라벨 번호 → 순열 매핑 (예측시 사용)
    label_to_perm = {idx: perm for idx, perm in enumerate(all_permutations)}
    
    print(f"총 {len(all_permutations)}개의 순열 클래스 생성")
    print("예시 매핑:")
    for i in range(5):
        print(f"  라벨 {i}: {all_permutations[i]}")
    
    return perm_to_label, label_to_perm

# 순열 매핑 생성
perm_to_label, label_to_perm = create_label_mappings()

총 24개의 순열 클래스 생성
예시 매핑:
  라벨 0: (0, 1, 2, 3)
  라벨 1: (0, 1, 3, 2)
  라벨 2: (0, 2, 1, 3)
  라벨 3: (0, 2, 3, 1)
  라벨 4: (0, 3, 1, 2)


In [3]:
def prepare_roberta_data(train_df, perm_to_label):
    """RoBERTa용 데이터 준비"""
    processed_data = []
    
    for _, row in train_df.iterrows():
        # 4개 문장 추출
        sentences = [row[f"sentence_{i}"] for i in range(4)]
        
        # 정답 순열
        answer_tuple = tuple([row[f"answer_{i}"] for i in range(4)])
        
        # 문장들을 [SEP]로 연결
        text = " [SEP] ".join(sentences)
        
        # 순열을 라벨로 변환
        label = perm_to_label[answer_tuple]
        
        processed_data.append({
            "text": text,
            "label": label,
            "original_sentences": sentences,
            "answer": answer_tuple
        })
    
    return processed_data

In [4]:
def augment_roberta_data(train_df, perm_to_label, multiplier=2):
    """데이터 증강으로 학습 데이터 늘리기"""
    augmented_data = []
    
    # 원본 데이터 처리
    original_data = prepare_roberta_data(train_df, perm_to_label)
    augmented_data.extend(original_data)
    
    # 증강 데이터 생성
    for _ in range(multiplier - 1):
        for _, row in train_df.iterrows():
            sentences = [row[f"sentence_{i}"] for i in range(4)]
            original_answer = [row[f"answer_{i}"] for i in range(4)]
            
            # 랜덤하게 문장 순서 섞기
            indices = list(range(4))
            np.random.shuffle(indices)
            
            # 새로운 순서의 문장들
            shuffled_sentences = [sentences[i] for i in indices]
            
            # 새로운 정답 계산
            new_answer = tuple([indices.index(original_answer[i]) for i in range(4)])
            
            # 텍스트 생성
            text = " [SEP] ".join(shuffled_sentences)
            label = perm_to_label[new_answer]
            
            augmented_data.append({
                "text": text,
                "label": label,
                "original_sentences": shuffled_sentences,
                "answer": new_answer
            })
    
    print(f"데이터 증강 완료: {len(original_data)} → {len(augmented_data)}")
    return augmented_data

print("데이터 전처리 함수 정의 완료!")

데이터 전처리 함수 정의 완료!


In [5]:
# 데이터 로드
print("데이터 로드 중...")
train_df = pd.read_csv('./train.csv')
print(f"원본 학습 데이터: {len(train_df)}개")

# 데이터 증강
print("\n데이터 증강 중...")
augmented_data = augment_roberta_data(train_df, perm_to_label, multiplier=3)

# 학습/검증 분할
print("\n학습/검증 데이터 분할...")
train_data, valid_data = train_test_split(
    augmented_data, 
    test_size=0.2, 
    random_state=42,
    stratify=[item["label"] for item in augmented_data]
)

print(f"학습 데이터: {len(train_data)}개")
print(f"검증 데이터: {len(valid_data)}개")

# 샘플 데이터 확인
print(f"\n샘플 데이터:")
print(f"텍스트: {train_data[0]['text'][:100]}...")
print(f"라벨: {train_data[0]['label']}")
print(f"정답: {train_data[0]['answer']}")

데이터 로드 중...
원본 학습 데이터: 7351개

데이터 증강 중...
데이터 증강 완료: 7351 → 22053

학습/검증 데이터 분할...
학습 데이터: 17642개
검증 데이터: 4411개

샘플 데이터:
텍스트: 따라서, 두 기술 모두를 체계적으로 훈련하는 것이 경기력 향상에 기여한다. [SEP] 효과적인 슈팅 기술은 득점 확률을 높이며, 상대 수비를 무너뜨리는 데 중요한 역할을 한다. [...
라벨: 23
정답: (3, 2, 1, 0)


In [6]:
print("모델 및 토크나이저 로드 중...")

# 모델 설정
model_name = "klue/roberta-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 방법 1: AutoTokenizer 사용 (가장 안전)
from transformers import AutoTokenizer, AutoModelForSequenceClassification

 # AutoTokenizer가 자동으로 올바른 토크나이저 클래스 선택
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"토크나이저 로드 성공: {type(tokenizer).__name__}")
    
# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=24,  # 24가지 순열
    cache_dir='C:/huggingface_cache'
)
print("✅ 모델 로드 성공!")

model.to(device)


print(f"모델: {model_name}")
print(f"디바이스: {device}")
print(f"파라미터 수: ~125M")
print(f"클래스 수: 24개 (4! 순열)")


모델 및 토크나이저 로드 중...
토크나이저 로드 성공: BertTokenizerFast


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 모델 로드 성공!
모델: klue/roberta-base
디바이스: cuda
파라미터 수: ~125M
클래스 수: 24개 (4! 순열)


In [7]:
print("데이터셋 토크나이징 중...")

# DataFrame으로 변환
train_df_processed = pd.DataFrame(train_data)
valid_df_processed = pd.DataFrame(valid_data)

# Dataset 객체 생성
train_dataset = Dataset.from_pandas(train_df_processed)
valid_dataset = Dataset.from_pandas(valid_df_processed)

# 토크나이징 함수
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=512
    )

# 토크나이징 실행
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

# 불필요한 컬럼 제거
train_dataset = train_dataset.remove_columns(["text", "original_sentences", "answer"])
valid_dataset = valid_dataset.remove_columns(["text", "original_sentences", "answer"])

print("토크나이징 완료!")
print(f"토크나이징된 학습 데이터: {len(train_dataset)}개")
print(f"토크나이징된 검증 데이터: {len(valid_dataset)}개")


데이터셋 토크나이징 중...


Map: 100%|██████████| 4411/4411 [00:00<00:00, 7160.72 examples/s]

토크나이징 완료!
토크나이징된 학습 데이터: 17642개
토크나이징된 검증 데이터: 4411개


In [ ]:
# 평가 메트릭 정의
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="./roberta_results",
    learning_rate=2e-5,
    

    per_device_train_batch_size=64,        
    per_device_eval_batch_size=128,         
    gradient_accumulation_steps=2,          
    
    num_train_epochs=20,
    
    # 평가 및 저장 (구버전 방식)
    eval_strategy="steps",                  
    eval_steps=150,                         
    save_strategy="steps",
    save_steps=150,                         
    save_total_limit=3,                    
    load_best_model_at_end=True,
    
    # 최적화 설정
    fp16=True,
    dataloader_pin_memory=True,
    dataloader_num_workers=6,               
    
    # 학습 최적화
    warmup_steps=150,                       
    weight_decay=0.01,
    
    # 로깅 (더 자주)
    logging_dir='./logs',
    logging_steps=20,                       
    report_to=None,
)

print("✅ 구버전 최적화된 TrainingArguments 설정 완료")

# 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n=== 🚀 최적화된 학습 설정 완료 ===")
print(f"배치 크기: {training_args.per_device_train_batch_size} ")
print(f"그래디언트 누적: {training_args.gradient_accumulation_steps} ")
print(f"실제 배치 크기: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps} ")
print(f"학습률: {training_args.learning_rate}")
print(f"에포크: {training_args.num_train_epochs}")
print(f"FP16: {training_args.fp16}")

# 메모리 사용량 체크 함수
def print_gpu_utilization():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        cached = torch.cuda.memory_reserved() / 1024**3
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        usage_percent = (allocated / total) * 100
        print(f"GPU 메모리: {allocated:.2f}GB 사용 / {total:.2f}GB 전체 ({usage_percent:.1f}% 사용)")
    else:
        print("CUDA 사용 불가")

print_gpu_utilization()

# 메모리 사용량 테스트
print("\n🧪 최적화된 설정 메모리 테스트 중...")
try:
    # 작은 테스트 배치로 메모리 사용량 확인
    sample_batch = next(iter(trainer.get_train_dataloader()))
    sample_batch = {k: v.to(model.device) for k, v in sample_batch.items()}
    
    model.eval()
    with torch.no_grad():
        outputs = model(**sample_batch)
    
    print("✅ 메모리 테스트 성공!")
    print_gpu_utilization()
    
    
except Exception as e:
    print(f"❌ 메모리 테스트 실패: {e}")

print(f"\n🎉 GPU 메모리 여유분을 활용한 최적화 완료!")
print(f"학습을 시작하세요! trainer.train()")

✅ 구버전 최적화된 TrainingArguments 설정 완료

=== 🚀 최적화된 학습 설정 완료 ===
배치 크기: 128 
그래디언트 누적: 2 
실제 배치 크기: 256 
학습률: 2e-05
에포크: 20
FP16: True
GPU 메모리: 1.70GB 사용 / 8.00GB 전체 (21.2% 사용)

🧪 최적화된 설정 메모리 테스트 중...
✅ 메모리 테스트 성공!
GPU 메모리: 1.70GB 사용 / 8.00GB 전체 (21.2% 사용)

🎉 GPU 메모리 여유분을 활용한 최적화 완료!
학습을 시작하세요! trainer.train()


In [ ]:
print("모델 학습 시작...")
print("=" * 50)

# 학습 실행
trainer.train()

print("\n학습 완료!")


모델 학습 시작...


Step,Training Loss,Validation Loss


In [ ]:
print("최종 검증 성능 평가...")

# 검증 데이터로 성능 평가
eval_results = trainer.evaluate()

print(f"\n=== 최종 성능 ===")
print(f"검증 정확도: {eval_results['eval_accuracy']:.4f} ({eval_results['eval_accuracy']*100:.2f}%)")
print(f"검증 손실: {eval_results['eval_loss']:.4f}")

# 상세 분석을 위한 샘플 예측
print(f"\n=== 샘플 예측 결과 ===")

# 몇 개 샘플에 대해 예측 결과 확인
sample_data = valid_data[:5]

for i, sample in enumerate(sample_data):
    # 예측
    inputs = tokenizer(
        sample['text'],
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_label = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_label].item()
    
    predicted_order = label_to_perm[predicted_label]
    true_order = sample['answer']
    
    print(f"\n샘플 {i+1}:")
    print(f"문장들: {sample['original_sentences']}")
    print(f"정답: {true_order}")
    print(f"예측: {predicted_order}")
    print(f"신뢰도: {confidence:.4f}")
    print(f"결과: {'✓ 정답' if predicted_order == true_order else '✗ 오답'}")

In [ ]:
print("\n모델 저장 중...")

# 모델과 토크나이저 저장
trainer.save_model("./roberta_results/final_model")
tokenizer.save_pretrained("./roberta_results/final_model")

print("모델 저장 완료!")
print("저장 위치: ./roberta_results/final_model")

In [ ]:
def predict_sentence_order(sentences, model, tokenizer, label_to_perm, device):
    """문장 순서 예측 함수"""
    # 입력 텍스트 생성
    text = " [SEP] ".join(sentences)
    
    # 토크나이징
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512
    ).to(device)
    
    # 예측
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_label = torch.argmax(predictions, dim=-1).item()
    
    # 라벨을 순열로 변환
    predicted_order = list(label_to_perm[predicted_label])
    confidence = predictions[0][predicted_label].item()
    
    return predicted_order, confidence

print("예측 함수 정의 완료!")


In [ ]:
print("테스트 데이터 예측 중...")

# 테스트 데이터 로드
test_df = pd.read_csv("./test.csv")
print(f"테스트 데이터: {len(test_df)}개")

predictions = []
confidences = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="예측 진행"):
    sentences = [row[f"sentence_{i}"] for i in range(4)]
    
    predicted_order, confidence = predict_sentence_order(
        sentences, model, tokenizer, label_to_perm, device
    )
    
    predictions.append(predicted_order)
    confidences.append(confidence)

# 평균 신뢰도 계산
avg_confidence = np.mean(confidences)
print(f"\n평균 예측 신뢰도: {avg_confidence:.4f}")

# 신뢰도 분포 확인
print(f"신뢰도 분포:")
print(f"  최소: {np.min(confidences):.4f}")
print(f"  최대: {np.max(confidences):.4f}")
print(f"  중간값: {np.median(confidences):.4f}")

In [ ]:
print("Submission 파일 생성 중...")

# 샘플 제출 파일 로드
sample_submission = pd.read_csv("./sample_submission.csv")

# 예측 결과를 제출 형식으로 변환
for i in range(4):
    sample_submission[f"answer_{i}"] = [pred[i] for pred in predictions]

# 파일 저장
submission_filename = "roberta_submission.csv"
sample_submission.to_csv(submission_filename, index=False)

print(f"제출 파일 저장 완료: {submission_filename}")

# 결과 요약 출력
print(f"\n" + "="*50)
print(f"🎯 RoBERTa 모델 학습 및 예측 완료!")
print(f"📊 검증 정확도: {eval_results['eval_accuracy']*100:.2f}%")
print(f"🔮 평균 예측 신뢰도: {avg_confidence*100:.2f}%")
print(f"📁 제출 파일: {submission_filename}")
print(f"="*50)